# Demo: Serenity Portfolio Analytics API

Basic demonstration of how to run portfolio performance statistics and Brinson attribution on a rich portfolio.

## Setup

In [ ]:
%%capture --no-stderr --no-display
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from serenity_sdk.widgets import ConnectWidget

# if you want to auto-connect, set this environment variable to your desired default
connect_widget = ConnectWidget(os.getenv('SERENITY_CONFIG_ID', None))

In [ ]:
from datetime import datetime

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# set Seaborn style
sns.set(style="whitegrid")

# create an alias to the api
api = connect_widget.get_api()

In [ ]:
from serenity_types.valuation.core import (
    AssetPosition,
    AssetWeight,
    PortfolioTimeseriesAndTrades,
    PositionTimeseries,
    RebalancingFrequency,
    Trades,
    Transfers,
    CompoundingFrequencyInput, ReturnsType
)
from serenity_types.valuation.portfolio_analytic import (
    BrinsonAttributionRequest,
    PortfolioAnalyticRequest,
    PortfolioFromAllocationRequest,
    PortfolioFromTradesRequest,
    PortfolioCompositionAndTrades, PortfolioComposition, CustomizedPortfolioComposition,
    PortfolioByMetadataIdOrCompositionValue
)
from serenity_types.pricing.core import MarkTime

asset_master = api.refdata().load_asset_master()
btc_asset_id = asset_master.get_asset_id_by_symbol('BTC')
eth_asset_id = asset_master.get_asset_id_by_symbol('ETH')
ada_asset_id = asset_master.get_asset_id_by_symbol('ADA')
bnb_asset_id = asset_master.get_asset_id_by_symbol('BNB')
usdc_asset_id = asset_master.get_asset_id_by_symbol('USDC')
usd_asset_id = asset_master.get_asset_id_by_symbol('USD')

## Creating portfolios

The new portfolio analytics API supports a much richer representation of portfolios, and has helper mechanisms that let you create portfolios from different types of inputs:

- initial cash position plus asset allocation rules
- initial positions, trades and transfers

### From allocations

In the first case you "trade" an initial cash position by periodically rebalancing the portfolio so the percentage weights are preserved. In the second case you start with a set of asset positions, possibly empty, and trade and transfer assets in and out, updating positions over time. In both cases the output will be timeseries of positions and trades.

In [ ]:
request = PortfolioFromAllocationRequest(
    initial_weights = [
        AssetWeight(asset_id=btc_asset_id, weight=0.4),
        AssetWeight(asset_id=eth_asset_id, weight=0.3),
        AssetWeight(asset_id=ada_asset_id, weight=0.2),
        AssetWeight(asset_id=bnb_asset_id, weight=0.1)
    ],
    initial_cash_quantity=1_000_000,
    start_datetime=datetime(2022, 1, 1, 0, 0, 0),
    end_datetime=datetime(2022, 12, 31, 0, 0, 0),
    rebalancing_frequency=RebalancingFrequency.DAILY
)
resp_obj = api.portfolio_analytics().create_portfolio_from_allocation(request)
pf_from_allocation_result = resp_obj.result
asset1 = asset_master.get_symbol_by_id(pf_from_allocation_result.positions.asset_ids[0], symbology='SERENITY')
asset2 = asset_master.get_symbol_by_id(pf_from_allocation_result.positions.asset_ids[1], symbology='SERENITY')
asset3 = asset_master.get_symbol_by_id(pf_from_allocation_result.positions.asset_ids[2], symbology='SERENITY')
asset4 = asset_master.get_symbol_by_id(pf_from_allocation_result.positions.asset_ids[3], symbology='SERENITY')
df = pd.DataFrame(pf_from_allocation_result.positions.quantities, columns=[asset1, asset2, asset3, asset4])
df['As Of Time'] = pd.to_datetime(pf_from_allocation_result.positions.as_of_times)
df.set_index('As Of Time', inplace=True)

# Create a line chart using Seaborn
plt.figure(figsize=(10, 6))
sns.lineplot(data=df, markers=True)

# Set labels and title
plt.title('Index Composition')

# Show the plot
plt.show()

### From trades

This variation creates a portfolio from several different combinations of inputs:

- positions, no trades or transfers: this takes current positions back through time, held constant
- positions, with trades and transfers: this builds up a portoflio over time from transactions

In [ ]:
trades = Trades(
    trade_datetime=[datetime(2022, 1, 2, 0, 0, 0)],
    base_asset_id=[btc_asset_id],
    quote_asset_id=[usdc_asset_id],
    quantity=[2.5],
    fill_price=[20000.1],
    commission_in_usd=[3.0]
)
transfers = Transfers(
    transfer_datetime=[datetime(2022, 1, 20, 0, 0, 0)],
    asset_id=[eth_asset_id],
    quantity=[5]
)
request = PortfolioFromTradesRequest(
    initial_positions=[ AssetPosition(asset_id=btc_asset_id, quantity=1),
                        AssetPosition(asset_id=eth_asset_id, quantity=10) ],
    start_datetime=datetime(2022, 1, 1, 0, 0, 0),
    end_datetime=datetime(2022, 12, 31, 0, 0, 0),
    trades=trades,
    transfers=transfers
)
resp_obj = api.portfolio_analytics().create_portfolio_from_trades(request)
pf_from_trades_result = resp_obj.result

assets = [ asset_master.get_symbol_by_id(x, symbology='SERENITY') if x != usd_asset_id else "USD" for x in pf_from_trades_result.positions.asset_ids ]
df = pd.DataFrame(pf_from_trades_result.positions.quantities, columns=assets)
df['As Of Time'] = pd.to_datetime(pf_from_trades_result.positions.as_of_times)
df.set_index('As Of Time', inplace=True)

# Create a line chart using Seaborn
plt.figure(figsize=(10, 6))
sns.lineplot(data=df.drop(["tok.usdc.ethereum", "USD"], axis=1), markers=True)

# Set labels and title
plt.title('Position History')

# Show the plot
plt.show()

## Creating Benchmarks

Create benchmarks the same way as creating portfolios

### Benchmark 1 - Specify by quantity

In [ ]:
request = PortfolioFromAllocationRequest(
    initial_weights = [
        AssetWeight(asset_id=btc_asset_id, weight=0.7),
        AssetWeight(asset_id=eth_asset_id, weight=0.3),
    ],
    initial_cash_quantity=1_000_000,
    start_datetime=datetime(2022, 1, 1, 0, 0, 0),
    end_datetime=datetime(2022, 12, 31, 0, 0, 0),
    rebalancing_frequency=RebalancingFrequency.DAILY
)
resp_obj = api.portfolio_analytics().create_portfolio_from_allocation(request)
bm1_by_quantity = resp_obj.result.positions.dict()

### Benchmark 2 - Specify by weight

In [ ]:
from copy import deepcopy

bm2_by_weight = deepcopy(bm1_by_quantity)
bm2_by_weight['weights'] = [[0.7, 0.3]] * len(bm2_by_weight['as_of_times'])
bm2_by_weight['quantities'] = None

### Benchmark 3 - Specify by weight, custom asset id, and custom prices

In [ ]:
import random

bm3_by_weight_custom_asset_prices = deepcopy(bm2_by_weight)
bm3_by_weight_custom_asset_prices['transient_asset_ids'] = ['my eth id', 'my btc id']
bm3_by_weight_custom_asset_prices['prices'] = [
    [1900 * random.uniform(1, 1.2), 30000 * random.uniform(1, 1.2)]
    for i in range(len(bm3_by_weight_custom_asset_prices['as_of_times']))
]
del bm3_by_weight_custom_asset_prices['asset_ids']

## Computing performance statistics

In [ ]:
pf_trades_and_transfers = PortfolioCompositionAndTrades(
    portfolio_composition=PortfolioByMetadataIdOrCompositionValue(
        positions=PortfolioComposition(**pf_from_allocation_result.positions.dict())
    ),
    trades=pf_from_allocation_result.trades,
    transfers=None
)
sim_trades = Trades(
    trade_datetime=[datetime(2022, 1, 10, 0, 0, 0)],
    base_asset_id=[btc_asset_id],
    quote_asset_id=[usdc_asset_id],
    quantity=[-5],
    fill_price=[20100.5],
    commission_in_usd=[6.0]
)
request = PortfolioAnalyticRequest(
    portfolio=pf_trades_and_transfers,
    benchmarks={
        "bm1_by_quantity": PortfolioByMetadataIdOrCompositionValue(
            positions=PortfolioComposition(**bm1_by_quantity)
        ),
        "bm2_by_weight": PortfolioByMetadataIdOrCompositionValue(
            positions=PortfolioComposition(**bm2_by_weight)
        ),
        "bm3_by_weight_custom_asset_prices": PortfolioByMetadataIdOrCompositionValue(
            positions=CustomizedPortfolioComposition(**bm3_by_weight_custom_asset_prices)
        ),
    },
    simulated_trades=sim_trades,
    mark_time=MarkTime.UTC,
    compounding_frequency=CompoundingFrequencyInput.DAILY,
    returns_type=ReturnsType.SIMPLE
)

resp_performance = api.risk_analytics().compute_portfolio_statistics(request)

## Computing performance attribution (Brinson)

In [ ]:
pf_trades_and_transfers = PortfolioCompositionAndTrades(
    portfolio_composition=PortfolioByMetadataIdOrCompositionValue(
        positions=PortfolioComposition(**pf_from_allocation_result.positions.dict())
    ),
    trades=pf_from_allocation_result.trades,
    transfers=None
)
sim_trades = Trades(
    trade_datetime=[datetime(2022, 1, 10, 0, 0, 0)],
    base_asset_id=[btc_asset_id],
    quote_asset_id=[usdc_asset_id],
    quantity=[-5],
    fill_price=[10100.5],
    commission_in_usd=[6.0]
)
request = BrinsonAttributionRequest(
    portfolio=pf_trades_and_transfers,
    benchmarks={
        "bm1_by_quantity": PortfolioByMetadataIdOrCompositionValue(
            positions=PortfolioComposition(**bm1_by_quantity)
        ),
        "bm2_by_weight": PortfolioByMetadataIdOrCompositionValue(
            positions=PortfolioComposition(**bm2_by_weight)
        ),
        "bm3_by_weight_custom_asset_prices": PortfolioByMetadataIdOrCompositionValue(
            positions=CustomizedPortfolioComposition(**bm3_by_weight_custom_asset_prices)
        ),
    },
    simulated_trades=sim_trades,
    mark_time=MarkTime.UTC,
    compounding_frequency=CompoundingFrequencyInput.DAILY,
    returns_type=ReturnsType.SIMPLE
)

resp_attribtuion = api.risk_analytics().compute_portfolio_performance_attribution(request)